In [1]:
!pip install -q hydra-core

     |████████████████████████████████| 133kB 4.2MB/s 
     |████████████████████████████████| 112kB 17.4MB/s 
     |████████████████████████████████| 645kB 40.5MB/s 


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 3.1MB/s 


In [ ]:
!pip install transformers

In [4]:
import torch
camembert = torch.hub.load('pytorch/fairseq', 'camembert')

from transformers import CamembertModel, CamembertTokenizer, CamembertForSequenceClassification, pipeline, AdamW
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from tqdm import tqdm, trange
import pandas as pd
import numpy as np

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp


/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/fairseq
creating build/temp.linux-x86_64-3.7/fairseq/clib
creating build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.7/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE="_gcc" -DPYBIND11_STDLIB="_libstdcpp" -DPYBIND11_BUILD_ABI="_cxxabi1011" -DTORCH_EXTENSION_

100%|██████████| 1012630426/1012630426 [01:43<00:00, 9822166.53B/s]


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
camembert = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.weight'

In [10]:
model_path = "/content/drive/MyDrive/trustpilot/model/camembert_sentiment_anal.pt"
df = pd.read_csv('drive/My Drive/trustpilot/data/trustpilot_to_predict.csv')

In [8]:
# camembert = torch.nn.DataParallel(camembert)

KeyboardInterrupt: ignored

In [ ]:
camembert.load_state_dict(torch.load(model_path), strict=False)
camembert.to(device)
camembert.eval()

In [12]:
MAX_LEN = 128
batch_size = 16

In [21]:
#Test the model on a comment
comments = ["j'aime le kebab", "le temps de livraison etait trop long", "je suis content"]

In [22]:
# comments = df['review']

In [23]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True, truncation=True, max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [24]:
# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  camembert(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())


In [25]:
for i in range(len(flat_pred)):
    print('Comment: ', comments[i])
    print('Label', flat_pred[i])

Comment:  j'aime le kebab
Label 1
Comment:  le temps de livraison etait trop long
Label 0
Comment:  je suis content
Label 1


In [18]:
df['sentiment'] = flat_pred

In [19]:
df

,review,rating,day,month,year,sentiment
0,"excellent ,site sérieux,professionnel et a l’é...",5,21.0,mars,2021.0,1
1,"expédition rapide, proposition de prix attract...",5,20.0,mars,2021.0,1
2,livrer très rapidement après commande emballag...,5,19.0,mars,2021.0,1
3,equipe efficace pour le service après vente ré...,5,19.0,mars,2021.0,1
4,vite correct ras,5,19.0,mars,2021.0,1
...,...,...,...,...,...,...
775,"premier achat sur le site, prix très attractif...",5,6.0,mars,2021.0,1
776,tout simplement les bests pour les imprimantes,5,5.0,mars,2021.0,1
777,achat et conseils téléphoniques top !,5,1.0,mars,2021.0,1
778,"les délais sont tenus, bien livrés dans les te...",5,5.0,mars,2021.0,1


In [20]:
df.to_csv('drive/My Drive/trustpilot/data/trustpilot_predicted.csv')